In [19]:
import os
import shutil
import yaml

CURRENT_DIR = os.path.join(os.getcwd(), "generate_datasets") # yolov8_training/notebooks/generate_datasets
print("Current directory: {}".format(CURRENT_DIR))
BASE_DIR = os.path.abspath(os.path.join(CURRENT_DIR, "..", "..", "..")) # yolov8_training
print("Base directory: {}".format(BASE_DIR))
DATASETS_DIR = os.path.join(BASE_DIR, "datasets") # yolov8_training/datasets
print("Dataset directory: {}".format(DATASETS_DIR))

os.makedirs(DATASETS_DIR, exist_ok=True)

RAW_CLASSES_DIR = os.path.join(DATASETS_DIR, "raw_all_classes") # yolov8_training/datasets/raw_all_classes
print("Raw classes directory: {}".format(RAW_CLASSES_DIR))
BANKNOTE_DIR = os.path.join(DATASETS_DIR, "banknote") # yolov8_training/datasets/banknote
print("Banknote directory: {}".format(BANKNOTE_DIR))

os.makedirs(BANKNOTE_DIR, exist_ok=True)

for split in ["train", "val", "test"]:
    for sub in ["images", "labels"]:
        os.makedirs(os.path.join(BANKNOTE_DIR, split, sub), exist_ok=True)

raw_yaml_path = os.path.join(RAW_CLASSES_DIR, "data.yaml")
if not os.path.exists(raw_yaml_path):
    raise RuntimeError(f"Missing raw dataset YAML: {raw_yaml_path}")
with open(raw_yaml_path, "r") as f:
    raw_data = yaml.safe_load(f)

banknote_names = {i: name for i, name in enumerate(raw_data["names"]) if "note" in name.lower()}
print(f"🎯 Banknote classes found: {list(banknote_names.values())}")

def filter_note_labels(label_path):
    lines = []
    with open(label_path, "r") as f:
        for line in f:
            cls, *rest = line.strip().split()
            cls = int(cls)
            name = raw_data["names"][cls]
            if "note" in name.lower():
                new_cls = list(banknote_names.keys()).index(cls)
                lines.append(f"{new_cls} {' '.join(rest)}\n")
    return lines

for split in ["train", "valid", "test"]:
    raw_images = os.path.join(RAW_CLASSES_DIR, split, "images")
    raw_labels = os.path.join(RAW_CLASSES_DIR, split, "labels")
    out_split = "val" if split == "valid" else split
    out_images = os.path.join(BANKNOTE_DIR, out_split, "images")
    out_labels = os.path.join(BANKNOTE_DIR, out_split, "labels")

    print(f"\n📁 Processing split: {split} -> {out_split}")

    for fname in os.listdir(raw_labels):
        label_path = os.path.join(raw_labels, fname)
        mapped_lines = filter_note_labels(label_path)
        if not mapped_lines:
            # print(f"⚠ Skipping {fname}: no banknote labels")
            continue
        shutil.copy(os.path.join(raw_images, fname.replace(".txt", ".jpg")), out_images)
        with open(os.path.join(out_labels, fname), "w") as f:
            f.writelines(mapped_lines)
        # print(f"✅ Processed {fname}")

note_yaml = {
    "train": "datasets/banknote/train/images",
    "val": "datasets/banknote/val/images",
    "test": "datasets/banknote/test/images",
    "nc": len(banknote_names),
    "names": {i: name for i, name in enumerate(banknote_names.values())}
}

yaml_path = os.path.join(BANKNOTE_DIR, "data.yaml")
with open(yaml_path, "w") as f:
    yaml.dump(note_yaml, f)

print(f"\n🎉 Banknote dataset generated at {BANKNOTE_DIR}")
print(f"📄 data.yaml location: {yaml_path}")

Current directory: D:\CurrencyDetectorApp\yolov8_training\notebooks\generate_datasets\generate_datasets
Base directory: D:\CurrencyDetectorApp\yolov8_training
Dataset directory: D:\CurrencyDetectorApp\yolov8_training\datasets
Raw classes directory: D:\CurrencyDetectorApp\yolov8_training\datasets\raw_all_classes
Banknote directory: D:\CurrencyDetectorApp\yolov8_training\datasets\banknote
🎯 Banknote classes found: ['1000_note', '100_note', '10_note', '2000_note', '200_note', '500_note', '50_note']

📁 Processing split: train -> train

📁 Processing split: valid -> val

📁 Processing split: test -> test

🎉 Banknote dataset generated at D:\CurrencyDetectorApp\yolov8_training\datasets\banknote
📄 data.yaml location: D:\CurrencyDetectorApp\yolov8_training\datasets\banknote\data.yaml
